# Calculating Zonal Statistics for Regions using WorldPop Data

## Google Earth Engine Method

### Preparation

In preparation, log into the Google Earth Engine GUI at https://code.earthengine.google.com and upload a shapefile or csv containing polygon boundaries of the regions for which you wish to calculate zonal statistics. For example, level 1 administrative boundaries for a country.

Once uploaded, these polygons will be available to use within the Google Earth Engine platform as a data 'asset', and will have a unique table identifier. For example, if you upload a shapefile named 'mdg_admn_ad1_py_s0_govn_pp_regions.shp', the identifier will be something either of the below:

- "projects/mapaction-cate/assets/mdg_admn_ad1_py_s0_govn_pp_regions"
- "users/cate/assets/mdg_admn_ad1_py_s0_govn_pp_regions"

### Step 1: Set up libaries and initialise the Earth Engine API.

In [14]:
import time
import ee
import pandas as pd

In [23]:
# Credentials to authorise Earth Engine
ee.Authenticate()
ee.Initialize()

### Step 2: Access WorldPop data

You can use either the constrained or unconstrained version.

In [6]:
# Get Worldpop Data

def get_worldpop_data(constrained=True):
    
    if constrained:
        return ee.ImageCollection("WorldPop/GP/100m/pop_age_sex_cons_unadj")
    else:
        return ee.ImageCollection("WorldPop/GP/100m/pop_age_sex")

In [7]:
pop_data = get_worldpop_data(constrained=False)
pop_data_img = pop_data.mosaic()

### Step 3: Access your polygons.

Replace the string here with your own asset id.

In [8]:
# Example
# polygons = ee.FeatureCollection("projects/mapaction-cate/assets/mdg_admn_ad1_py_s0_govn_pp_regions")

polygons = ee.FeatureCollection("YOUR_ASSET_ID_HERE")

### Step 4: Use reduce region to perform zonal stats

In [9]:
stats = pop_data_img.reduceRegions(reducer=ee.Reducer.sum(), collection=polygons, scale=95)

### Step 5: Export csv




In [16]:
task = ee.batch.Export.table.toDrive(stats,
                                     description='zonal-stats-results',
                                     folder='zonal_stats_exports',
                                     fileNamePrefix=None,
                                     fileFormat='csv',
                                     selectors=None)
task.start()

while task.status()['state'] in ['READY', 'RUNNING']:
    print('Export task status is', task.status()['state'])
    time.sleep(5)
else:
    print(task.status()['state'])

Export task status is  READY
Export task status is  READY
Export task status is  READY
Export task status is  READY
Export task status is  READY
Export task status is  RUNNING
{'state': 'COMPLETED', 'description': 'zonal-stats-results', 'creation_timestamp_ms': 1644847939938, 'update_timestamp_ms': 1644847972262, 'start_timestamp_ms': 1644847966334, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1sFytWJ1YfA3xiVq9ubjYdyLllQ9nqwX6'], 'attempt': 1, 'id': 'CYI7RC5TA73OEME6G6J2VSLG', 'name': 'projects/earthengine-legacy/operations/CYI7RC5TA73OEME6G6J2VSLG'}


### Step 6: Open csv and format everything nicely

Earth Engine has now exported a csv to your Google Drive in a folder 'zonal_stats_export'. Locate and download this file to your computer.

In [17]:
# Example
# downloaded_csv_path = "/Users/cate/Downloads/zonal-stats-results.csv"

# Add your path here
downloaded_csv_path = "YOUR/PATH/HERE.csv"

raw_csv = pd.read_csv(downloaded_csv_path)

In [18]:
columns_to_remove = ['ADM0_EN', 'ADM0_PCODE', 'ADM1_TYPE', 'system:index', '.geo', 'PROV_TYPE', 'PROV_CODE', 'SOURCE']
df = raw_csv.drop(columns_to_remove, axis=1)

In [19]:
female_pops = [i for i in list(df.columns) if i.startswith('F_')]
male_pops = [i for i in list(df.columns) if i.startswith('M_')]

df['totalF'] = df[female_pops].sum(axis=1)
df['totalM'] = df[male_pops].sum(axis=1)

In [20]:
df.insert(2, 'totalM', df.pop('totalM'))
df.insert(2, 'totalF', df.pop('totalF'))
df.insert(2, 'totalPop', df.pop('population'))
df.insert(2, 'OLD_PROVIN', df.pop('OLD_PROVIN'))

In [21]:
df = df.round(decimals=2)

In [22]:
# Example
# final_results_filename = "mdg_populations_adm1" 

# Add your output filename here
final_results_filename = "YOUR_FILENAME_HERE" 

df.to_csv(final_results_filename + ".csv")

### Step 7: Sanity check

In [87]:
df

,ADM1_EN,ADM1_PCODE,OLD_PROVIN,totalPop,totalF,totalM,F_0,F_1,F_10,F_15,...,M_40,M_45,M_5,M_50,M_55,M_60,M_65,M_70,M_75,M_80
0,Analamanga,MG11,Antananarivo,4497255.87,2324930.21,2172325.64,52450.21,220830.14,236798.48,247779.00,...,124079.83,96830.87,235206.86,71827.34,59937.16,45350.23,32641.68,19090.84,15502.95,8879.73
1,Vakinankaratra,MG12,Antananarivo,1787336.81,902407.07,884929.73,26836.74,100833.55,112204.67,98631.46,...,42776.85,36928.30,124326.11,23512.39,20207.94,16804.07,10569.10,4440.49,3881.33,3042.05
2,Itasy,MG13,Antananarivo,699797.69,348597.93,351199.76,10648.43,42137.38,46877.25,37881.47,...,17585.90,11910.21,47745.26,11118.38,7114.27,5098.38,4790.42,1695.10,1746.71,1712.18
3,Bongolava,MG14,Antananarivo,610676.54,303980.57,306695.97,8764.29,35534.06,40672.88,32734.61,...,14648.41,9712.06,44897.00,5995.29,6235.33,5724.67,5073.47,2706.83,955.58,930.35
4,Haute Matsiatra,MG21,Fianarantsoa,1150649.43,581563.51,569085.93,23125.24,73293.59,61702.86,53391.78,...,23094.52,15461.95,85838.71,14183.51,15301.15,6804.98,8180.47,3717.41,3650.96,2346.04
5,Amoron I Mania,MG22,Fianarantsoa,665998.74,336130.52,329868.22,9306.97,40528.55,43122.05,34188.40,...,13525.57,10497.92,47755.04,11358.55,9087.56,4758.54,3939.64,3307.27,2663.60,984.11
6,Vatovavy Fitovinany,MG23,Fianarantsoa,1800795.94,910823.45,889972.47,36279.29,122704.27,102512.61,93886.44,...,34956.51,32049.97,130398.94,30435.16,17452.60,15717.30,9588.51,4964.65,2956.52,1605.52
7,Ihorombe,MG24,Fianarantsoa,569144.73,284186.19,284958.54,13098.71,38029.98,32687.97,25580.68,...,13948.38,11176.93,40589.17,8007.32,5599.92,3629.96,2866.01,1592.35,1045.47,844.52
8,Atsimo Atsinanana,MG25,Fianarantsoa,1300250.24,673861.30,626388.94,28877.09,93771.35,84468.94,74474.28,...,23269.44,16934.66,86734.34,17929.23,12485.96,12704.65,5547.61,5557.90,1448.13,1647.34
9,Atsinanana,MG31,Toamasina,1228541.25,622875.80,605665.44,14973.51,59117.76,67689.07,71445.88,...,23821.84,21476.42,71591.85,21676.38,16988.36,11749.12,9947.08,4399.63,2890.93,2691.18
